# LightGBM: A Highly Efficient Gradient Boosting Decision Tree

2018.11.04. <br>
Jaehwi Park

## 1. Introduction

> - GBM의 성능이 뛰어나지만, 데이터의 크기가 커지며 Accuracy <-> Efficiency Trade-off 문제가 발생하기 시작함
> - 기존 GBM은 매번 모든 Data를 탐색해 Information Gain을 구해야 했음... 계산복잡도에 Feature 수뿐만 아니라 Data 수도 같이 고려됨
> - 본 논문에서는 두 가지 Novel Points를 제시함
>> GOSS(Gradient-Based One-Side Sampling)
>> - 더 큰 Gradient를 가진 Data Instances가 Information Gain에 더 많이 기여함
>> - 이러한 방식으로 랜덤 샘플링보다는 정교한 I.G. 예측이 가능해졌음
>>
>> EFB(Exclusive Feature Bundling)
>> - 실제 Application에서, Features의 숫자는 많지만 feature space는 상당히 sparse함
>> - 대부분의 그런 데이터는 제외할만 함
>> - 그리고 one-hot encoding된 변수들도 대부분 그러함
>> - 본 논문에서 안전하게 그러한 exclusive features를 한 데 묶는 방법을 제시함
>> - Bundling Problem을 Graph Coloring Problem으로 치환하며, Greedy Algorithm으로 해를 구함


## 2. Preliminaries
### 2.1 GBDT and Its Complexity Analysis

> - GBDT는 DT의 앙상블 모델이며, 차례차례 학습이 진행됨
> - 매 iteration에서 negative gradient(residual errors)를 적합하며 학습이 진행됨

---
> - DT 학습에서 가장 큰 시간 소모는 split-point를 찾는 것임
> - DT의 학습속도를 향상시키기위해 the pre-sorted algorithm 대신 the histogram-based algorithm이 사용됨
> - Histogram 생성에 #data$\times$#features, Split point 탐색에 #bin$\times$#features 이므로 최종 복잡도는 #data, #features에 달려있음


### 2.2 Related work
> - GBDT의 implementation은 여러가지가 있지만 성능이 가장 좋은 XGBoost를 Base model로 사용함

---
> - 데이터 샘플링이 속도를 증가시키기 위한 일반적인 방법이며, SGB 알고리즘은 특정 가중치 이하의 sample을 모두 filtering 함
> - 그러나 모든 데이터가 각각의 가중치를 갖고 있지 않으므로 GBDT에서 사용할 수는 없음
> - 주성분분석이나 사영추적 방법을 사용할 수도 있지만, 사라지는 변수의 영향도를 무시할 수 없음

## 3. Gradient-based One-Side Sampling (GOSS)
### 3.1 Algorithm Description

> - small-gradient의 의미는 에러가 작다는 의미이므로 해당 instance가 well-trained 됐다고 생각할 수 있음
> - 그러므로 gradient의 값이 큰 것들만 선택하면 되지만, 그러면 샘플 분포가 틀어질 수 있으므로 GOSS 방법을 제안함

![g](./gradient.png)

출처: http://www.ccs.neu.edu/home/vip/teach/MLcourse/4_boosting/slides/gradient_boosting.pdf

---
> - GOSS에서 gradient 값이 큰 instances는 모두 보관하고, gradient 값이 작은 instances에 대해서는 Random Sampling을 진행함
> - gradient 순으로 정렬한 상태에서 값이 큰 상위 a% instance는 모두 선택하고, 나머지 데이터 중 b%의 instances를 Random Sampling 함
> - 분포가 틀어지는 것을 막기 위해 Random Sample의 Information Gain 계산시, 상수 "(1-a)/b "를 곱해줌

### 3.2 Theoretical Analysis

>__definition 3.1__
>
> - sampling 된 데이터만으로 I.G를 계싼함
> - I.G. 계산할 때 low gradient 그룹은 (1-a)/b 값을 곱해줌
> - 이렇게 함으로써, under-trained data에 대해 더 집중할 수 있음
> - 그리고 샘플링 된 데이터로 계산하므로 연산 속도도 빨라지는 장점이 있음
>
> ![3_1](./definition3_1.png)


---
> __Theorem 3.2__
>
> hmmm...

## 4. Exclusive Feature Bundling

> - Sparse한 데이터가 너무 많으므로 잘 묶어줘야 함
> - 1) 어떤 변수들을 묶어줘야 하는지, 2) 묶는 방법이 두 가지 메인 이슈임

### Theorem 4.1 Bundling 문제는 Graph-Coloring 문제로 치환할 수 있음.. 그렇다고 함...

![CG](./graph_coloring.png)

> - 모든 features를 Graph의 Vertex로 생각하고, 서로 Mutually Exclusive가 아닐 경우에만 Edge를 연결
> - 그 상태에서 Greedy하게 문제를 해결하면 좋은 근사해가 됨
> - 또한, 100% M.E.는 아니지만 꽤나 많은 변수가 0이 아닌 값을 동시에 갖지 않음
> - 우리가 그러한 충돌을 조금 인정한다면, 변수의 수가 더 크게 줄어듦
> - 알고리즘 스텝은,
>> - 1) 모든 features간 edge를 연결하며 edge의 weight로 변수간 conflicts 값을 갖게 함
>> - 2) features를 다른 features들과 연결된 숫자인 degree 순으로 정렬함
>> - 3) 작은 conflict 값은 무시하며 greedy algorithm으로 기존 번들에 넣거나 새로운 번들을 만들어 냄
> - 본 작업은 전체 lightGBM 로직에서 가장 처음에 한 번만 하면 되고,
> - 연산복잡도는 (#features)^2 임

---
> 같은 번들 내 변수를 묶는 방법은,
> - offset 을 설정하여 값의 범위가 겹치지 않게 만들고
> - 원래 변수들을 새로운 값으로 대체함
> ![algorithm4](./algorithm4.png)

---
# Quick Start

In [2]:
import lightgbm as lgb

In [ ]:
# 1. Data pipeline

# 1.1 lgb를 위한 데이터 생성.. 
# libsvm text file(.svm) 또는 lightGBM binary(.bin)이 기본
# 그런데 그냥 txt도 가능
train_data = lgb.Dataset('train.txt')

# 1.2 lightGBM binary file로 저장해두면 속도가 빠르다고 함
train_data.save_binary('train.bin')

# 1.3 Validation set 
# 생성은
test_data = train_data.create_valid('test.svm')
test_data = lgb.Dataset('test.svm', reference=train_data)
# val set은 항상 준비돼있어야 한다고 함

# 1.4 Data feature 지정
# lightGBM에서 categorical은 그냥 써주라고 함. one-hot encoding이 필요 없고, 원상태로 두는게 8배정도 더 빠르다고 함
# 다만, categorical 데이터를 int 형식으로 미리 만들어야 함. 문자 안돼요!
train_data = lgb.Dataset(data, label=label, feature_name=['c1', 'c2', 'c3'], categorical_feature=['c3'])

In [ ]:
# 2. Parameter Setting

# 2.1 단일 지정
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
param['metric'] = 'auc'

# 2.2 복수 지정
param['metric'] = ['auc', 'binary_logloss']

In [ ]:
# 3. Training

# 3.1 학습
bst = lgb.train(param, train_data, num_round=10, valid_sets=[test_data])

# 3.2 저장
bst.save_model('model.txt')
json_model = bst.dump_model()

# 3.3 불러오기
bst = lgb.Booster(model_file='model.txt')

In [ ]:
# 4. Inference
ypred = bst.predict(data)